In [1]:
include("optim_lib.jl")
include("lib_func.jl")

# Constants and Parameters
d1, d2, ds = 10, 10, 1  # Degree range and step
const n, a, b = 2, 2, 10 
const C = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alph = .5 , 9 / 10  # Sampling parameters
f = tref # Objective function

tref (generic function with 1 method)

In [7]:
coeffs_poly_approx = main_gen(f, n, d1, d2, ds, delta, alph, C, 0.1)
coeffs_poly_precis = precise_gen(f, n, d1, d2, ds, delta, alph, C, 0.1)

dimension Vector space: 66
sample size: 729
dimension Vector space: 66
sample size: 729


1-element Vector{Vector{BigFloat}}:
 [1.163396863853211792393444463025765380254987769974704233637658564137226148884713, 0.08794815361798803194743677254502048381208877958083788106651677212839702996209218, -0.082035570920905304153729651735062090770951067987921054470919342504065311122715, 0.2713670219757378067816243157083952016817635912839023595445217956453260522563232, -0.07113466526678588274449848419104244162405784714636411969004963981550342296306271, -0.0781328685654424234865171447274294226031700901065300803076757301848493767959917, -0.03045695196712847785936389071031625962484552182295057806031375234941395919127915, -0.1537586009292908222689652826003441195379924808057757585227415401643231306512439, 0.0392607032272859444139790999714585680645738581026471605621421415339373836625261, 0.458363429688199085641268371049017768088086053376407038923021500462508029673575  …  -0.4604650893755408722260909791455069364261240629249595658982608680760197607310251, 3.20102699405426483462180378001755565832

In [3]:
using DynamicPolynomials, MultivariatePolynomials, HomotopyContinuation, ProgressLogging, DataFrames

include("hom_solve.jl") # Include the homotopy solver and main function
vars = @polyvar(x[1:n]) # Define polynomial ring 
h_x, h_y, col = main_2d(n, d1, d2, ds, coeffs_poly_approx) # main_2d is in hom_solve.jl
h_prec_x, h_prec_y, col_prec = main_2d(n, d1, d2, ds, coeffs_poly_precis) # Is the type going to work here?
df = DataFrame(x=C * h_x, y=C * h_y, col=col)
df[!, :result] = [f([df.x[i], df.y[i]]) for i in 1:nrow(df)];
df[!, :local_minima] = zeros(nrow(df))
df[!, :distance_to_minima] = zeros(nrow(df))
df[!, :steps] = zeros(Int, nrow(df))
df[!, :converged] = falses(nrow(df))

dh = DataFrame(x=C * h_prec_x, y=C * h_prec_y, col=col)
dh[!, :result] = [f([dh.x[i], dh.y[i]]) for i in 1:nrow(dh)];
dh[!, :local_minima] = zeros(nrow(dh))
dh[!, :distance_to_minima] = zeros(nrow(dh))
dh[!, :steps] = zeros(Int, nrow(dh))
dh[!, :converged] = falses(nrow(dh))



Computing mixed cells... 2    Time: 0:00:00
Computing mixed cells... 4    Time: 0:00:02
  mixed_volume:  81


Tracking 81 paths...   2%|▊                             |  ETA: 0:08:38



Tracking 81 paths... 100%|██████████████████████████████| Time: 0:00:14
  # paths tracked:                  81
  # non-singular solutions (real):  79 (79)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         79 (79)
64×8 DataFrame
 Row │ x           y           col    result      local_minima  distance_to_minima  steps  converged 
     │ Float64     Float64     Int64  Float64     Float64       Float64             Int64  Bool      
─────┼───────────────────────────────────────────────────────────────────────────────────────────────
   1 │  0.16124     0.0795496     10   1.95303             0.0                 0.0      0      false
   2 │ -0.0282199  -0.0980983     10  -1.26667             0.0                 0.0      0      false
   3 │ -0.100347   -0.0420769     10   3.97242             0.0                 0.0      0      false
   4 │  0.0368614  -0.190228      10   3.09569             0

In [6]:
println(sort(df, :result))
println(sort(dh, :result))

64×8 DataFrame
 Row │ x           y           col    result      local_minima  distance_to_minima  steps  converged 
     │ Float64     Float64     Int64  Float64     Float64       Float64             Int64  Bool      
─────┼───────────────────────────────────────────────────────────────────────────────────────────────
   1 │ -0.0262432   0.130529      10  -2.87678             0.0                 0.0      0      false
   2 │ -0.0282199  -0.0980983     10  -1.26667             0.0                 0.0      0      false
   3 │  0.0989931   0.125441      10  -1.23555             0.0                 0.0      0      false
   4 │ -0.183384   -0.190557      10  -1.02371             0.0                 0.0      0      false
   5 │  0.19031    -0.0967601     10  -0.965203            0.0                 0.0      0      false
   6 │ -0.0211601   0.0277566     10  -0.838417            0.0                 0.0      0      false
   7 │ -0.0221142   0.0652768     10  -0.836453            0.0           

Initialize Zeroth order optimization method at each of the critical points of the approximants. 
Record the distance between critical point and output of LBFGS.

In [4]:
using Optim
for i in 1:nrow(df)
    try
        println("Optimizing for point $i")
        x0 = [df.x[i], df.y[i]]
        res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
        minimizer = Optim.minimizer(res)
        min_value = Optim.minimum(res)
        steps = res.iterations
        converged = Optim.converged(res)
        distance = norm(x0 - minimizer) 
        df.local_minima[i] = min_value
        df.distance_to_minima[i] = distance
        df.steps[i] = steps
        df.converged[i] = converged
    catch e
        println("Error: $e")
    end
end


UndefVarError: UndefVarError: `df` not defined

In [5]:
# filtered_df = df[df.col.==6, :];
sorted_df = sort(filtered_df, :local_minima)

UndefVarError: UndefVarError: `filtered_df` not defined

Plot all the critical points on the contour plot of the objective, colored by degree.

In [6]:
using PlotlyJS, Colors

# Generate the grid and evaluate the function
N = 150  # resolution of the grid
x = range(-C, C, length=N)
y = range(-C, C, length=N)
z = [f([xi, yi]) for yi in y, xi in x]
# Generate a color palette based on the number of unique `col` values
unique_cols = unique(df.col)
num_colors = length(unique_cols)
color_palette = distinguishable_colors(num_colors)
# Map the `col` values to the corresponding colors
col_to_color = Dict(unique_cols .=> color_palette)
# Create individual scatter traces for each unique `col` value
scatter_traces = [scatter(x=df[df.col .== c, :x], y=df[df.col .== c, :y], mode="markers", marker=attr(color=col_to_color[c], size=5), name="Degree $c") for c in unique_cols]
# Create the contour plot
cp = contour(x=x, y=y, z=z, ncontours=80, colorscale="Viridis", showscale=false)
# Combine contour plot and scatter traces
all_traces = [cp; scatter_traces...]
# Customize layout to handle legend groups
layout = Layout(
    title="Scatter and Contour Plot",
    xaxis_title="X-axis",
    yaxis_title="Y-axis",
    legend=(tracegroupgap=10, groupclick="toggleitem"),
    height=800 # Increase the height to make room for the legend 
)
# Display the combined plot with legend
display(plot(all_traces, layout))

Doesn't make too much sense, but maybe we could plot the level sets of the polynomial approximants

In [ ]:
# plot on subdomain 
SmC = 0.5 * C
x = range(-C, C, length=N)
y = range(-C, C, length=N)
# Generate the grid and evaluate the function
zi = [f([xi, yi]) for yi in y, xi in x]
# Create the plot
sf = surface(x=x, y=y, z=zi)
# Layout for the plot
layout = Layout(title="3D Plot of f(x, y) = sin(πx) * sin(πy)",
    scene=attr(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="f(x, y)"
    ))
# Display the plot
plot(sf, layout)

In [ ]:
println(tref)